# MCP

可以将 MCP 想象成 AI 应用程序的"USB-C端口"，提供了一种将AI模型连接到不同数据源和工具的标准化方法。

传统上，M个AI应用程序连接到N个工具/服务需要 M×N 个自定义集成。而MCP能将复杂的 M×N 问题转化为更易管理的 M+N 问题，每个主机和服务器只需要实现一次MCP标准

## MCP 架构组件
### 主机 (Host)
定义：终端用户直接交互的面向用户的AI应用程序
举例：
- Anthropic的Claude Desktop
- Cursor等AI增强的IDE

### 客户端 (Client)
定义：主机应用程序中管理与特定MCP服务器通信的组件
举例：在Claude Desktop中，负责与MCP服务器交互的模块，每个客户端与单个服务器保持1:1连接

### 服务器 (Server)
定义：通过MCP协议公开功能（工具、资源、提示）的外部程序或服务
举例：
- 文件系统访问服务器（提供文件读写功能）
- 数据库查询服务器（提供SQL查询功能）
- 12306车票查询服务器（提供火车票查询功能）

![image.png](https://miro.medium.com/v2/resize:fit:1400/format:webp/1*LtZYdQvtDLzN4CwmlHvfFA.png)

## MCP实现

## 第三方MCP调用